## Install the library

In [ ]:
!pip install geemap
!pip install ee

## Import the Libraries

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets
import pandas as pd
import numpy as np
from PIL import Image


## Authenticate and Initialise with your account associated with Google Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize()

## Data Range

In [ ]:
start_date = '2022-01-01'
end_date = '2022-06-30'

# Finding water quality parameters

## Sentinel-2A

### Rastar to Vector Conversion

In [ ]:
Map = geemap.Map()

geometry = ee.Geometry.Point([77.2052, 23.1443])
image = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(geometry) \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20)) \
        .first()
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B8'])

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1= ndwiMasked.toInt()
vectors = ndwiMasked1.reduceToVectors(scale = 30.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)


In [ ]:
Map.addLayer(ndwi, {}, 'NDWI')
Map.addLayer(ndwiMasked1, {'palette': ['00FFFF', '0000FF']}, 'Water')
Map.centerObject(geometry, zoom=12)

In [ ]:
Map

In [ ]:
# Upper Lake, Bhopal

geometry = ee.Geometry.Point([77.2052, 23.1443])



Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
            .filterDate(start_date,end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
            .median()


mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')

ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')


ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')

dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')


# Visualise

## MNDWI - Modified Normalised Diffrence Water Index <br>
<h4>The Modified Normalized Difference Water Index (MNDWI) uses green and SWIR bands for the enhancement of open water features. It also diminishes built-up area features that are often correlated with open water in other indices.</h4>

In [ ]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(mndwi, {'min': -1, 'max': 1, 'palette': ['white', 'black', 'blue']}, 'MNDWI')
Map

## PH - Power Of Hydrogen
<h4><b>Formula: </b>pH = 8.339 - 0.827 * (B1/B8)</h4>


In [ ]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ph, {'min': 6, 'max': 9, 'palette': ['red', 'green', 'purple']}, 'pH')
Map

## NDTI - Normalize Difference Turbidity Index
<h4>The Normalize Difference Turbidity Index (NDTI) which is estimated using the spectral reflectance values of the water pixels is used to estimate the turbidity in water bodies. It uses the phenomenon that the electromagnetic reflectance is higher in green spectrum than the red spectrum for clear water.</h4>

In [ ]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndti, {'min': -1, 'max': 1, 'palette': ['black', 'yellow', 'red']}, 'NDTI')
Map

## NDCI - Normalised Diffrence Chlorophyll Index
<h4>NDCI is an index that aims to predict the plant chlorophyll content. It is calculated using the red spectral band B04 with the red edge spectral band B05</h4>

In [ ]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndci, {'min': -1, 'max': 1, 'palette': ['white', 'black','yellow']}, 'NDCI')
Map

## DO - Dissolved Oxygen 
<h4>Dissolved oxygen (DO) is the amount of oxygen that is present in water. Water bodies receive oxygen from the atmosphere and from aquatic plants. Running water, such as that of a swift moving stream, dissolves more oxygen than the still water of a pond or lake.</h4>

In [ ]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(dissolvedoxygen, {'min': 0, 'max': 20, 'palette': ['white', 'black', 'teal']}, 'Dissolved Oxygen')
Map

### Landsat-8 (Temperature)

In [ ]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate(start_date,end_date) \
.filterBounds(vectors).median()
temp  = col.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')


In [ ]:
Map.addLayer(temp, {'min': -30, 'max': 30, 'palette': ['red', 'white', 'black']}, 'Temperature')
Map